In [1]:
import os
import sys

# Sätt base_dir (projektets rotkatalog)
base_dir = os.path.abspath("..")  # En nivå upp från notebooks/
sys.path.append(base_dir)  # Lägg till base_dir i sys.path

# Nu fungerar importerna
from src.objects.optimizer import Optimizer
from src.objects.package import Package
from src.data_processing import load_data
import pandas as pd

lagerstatus = os.path.join(base_dir, "data", "lagerstatus.csv")

# Kontrollera att imports fungerar
print("Imports lyckades!")

Imports lyckades!


In [2]:
log_file = os.path.join(base_dir, "logs", "optimization.log")

# Rensa tidigare loggar
if os.path.exists(log_file):
    os.remove(log_file)

# Skapa testdataset
packages = load_data(lagerstatus)

print(f"Testdata skapad! {len(packages)}")



Testdata skapad! 10000


In [3]:
# Originalmetod: Ingen optimering, bara lista paketen
optimizer = Optimizer(packages, max_trucks=10, max_capacity=800)
optimizer.distribute_packages()
optimizer.display_results()
optimizer.reset()


Total remaining packages in storage: 7799

--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 799.2, Packages: 385)
Truck(Truck_2, Total Weight: 799.8000000000014, Packages: 269)
Truck(Truck_3, Total Weight: 799.6000000000001, Packages: 231)
Truck(Truck_4, Total Weight: 798.7999999999993, Packages: 214)
Truck(Truck_5, Total Weight: 799.9999999999993, Packages: 200)
Truck(Truck_6, Total Weight: 799.1, Packages: 195)
Truck(Truck_7, Total Weight: 799.8999999999992, Packages: 186)
Truck(Truck_8, Total Weight: 799.3000000000012, Packages: 179)
Truck(Truck_9, Total Weight: 799.3999999999985, Packages: 175)
Truck(Truck_10, Total Weight: 799.8999999999992, Packages: 167)

Totalt antal paket kvar i lager: 7799
Total Förtjänst (levererade paket): 14660.0
Totala Straffavgifter: -1484.0
Använda bilar: 10


In [4]:
# Greedy Optimering
optimizer_greedy = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_greedy.greedy_optimization()
optimizer_greedy.display_results()
optimizer_greedy.reset()



--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 799.2, Packages: 385)
Truck(Truck_2, Total Weight: 799.8000000000014, Packages: 269)
Truck(Truck_3, Total Weight: 799.6000000000001, Packages: 231)
Truck(Truck_4, Total Weight: 798.7999999999993, Packages: 214)
Truck(Truck_5, Total Weight: 799.9999999999993, Packages: 200)
Truck(Truck_6, Total Weight: 799.1, Packages: 195)
Truck(Truck_7, Total Weight: 799.8999999999992, Packages: 186)
Truck(Truck_8, Total Weight: 799.3000000000012, Packages: 179)
Truck(Truck_9, Total Weight: 799.3999999999985, Packages: 175)
Truck(Truck_10, Total Weight: 799.8999999999992, Packages: 167)

Totalt antal paket kvar i lager: 7799
Total Förtjänst (levererade paket): 14660.0
Totala Straffavgifter: -1484.0
Använda bilar: 10


In [5]:
# Genetisk Algoritm
optimizer_genetic = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_genetic.reset()
optimizer_genetic.genetic_optimization(population_size=15, generations=100, mutation_rate=0.1)


Generation: 1, Best Fitness: 2216394.0, Population Size: 15
Generation: 10, Best Fitness: 2216394.0, Population Size: 15
Generation: 20, Best Fitness: 2215392.0, Population Size: 15
Generation: 30, Best Fitness: 2212370.0, Population Size: 15
Generation: 40, Best Fitness: 2211352.0, Population Size: 15
Generation: 50, Best Fitness: 2214367.0, Population Size: 15
Generation: 60, Best Fitness: 2212353.0, Population Size: 15
Generation: 70, Best Fitness: 2211340.0, Population Size: 15
Generation: 80, Best Fitness: 2210335.0, Population Size: 15
Generation: 90, Best Fitness: 2205296.0, Population Size: 15
Generation: 100, Best Fitness: 2205296.0, Population Size: 15
Package Package(4230365328.0, Weight: 1.4, Profit: 2.0, Deadline: 4.0) not found in remaining_packages!
Package Package(4230365648.0, Weight: 1.9, Profit: 3.0, Deadline: 2.0) not found in remaining_packages!
Package Package(4230365328.0, Weight: 1.4, Profit: 2.0, Deadline: 4.0) not found in remaining_packages!

--- Resultat för

In [ ]:
# Greedy + Genetisk Algoritm
optimizer_combined = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_combined.greedy_optimization()  # Börja med greedy
optimizer_combined.genetic_optimization(population_size=10, generations=50, mutation_rate=0.1)  # Förbättra


Generation: 1, Best Fitness: 2216394.0, Population Size: 10
Generation: 10, Best Fitness: 2216394.0, Population Size: 10
Generation: 20, Best Fitness: 2213376.0, Population Size: 10
Generation: 30, Best Fitness: 2212373.0, Population Size: 10
Generation: 40, Best Fitness: 2210365.0, Population Size: 10
Generation: 50, Best Fitness: 2211367.0, Population Size: 10
Package Package(4230365328.0, Weight: 1.4, Profit: 2.0, Deadline: 4.0) not found in remaining_packages!

--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 799.2, Packages: 385)
Truck(Truck_2, Total Weight: 799.8000000000012, Packages: 269)
Truck(Truck_3, Total Weight: 797.4999999999998, Packages: 230)
Truck(Truck_4, Total Weight: 790.0999999999992, Packages: 212)
Truck(Truck_5, Total Weight: 799.5999999999996, Packages: 200)
Truck(Truck_6, Total Weight: 798.6000000000001, Packages: 195)
Truck(Truck_7, Total Weight: 798.6999999999996, Packages: 185)
Truck(Truck_8, Total Weight: 796.5000000000008, Packages: 178)
Truck(

In [7]:
# Läs loggfilen
log_data = []
with open(log_file, "r") as log:
    for line in log:
        if "Iteration" in line:
            try:
                parts = line.strip().split(", ")
                iteration = int(parts[0].split(": ")[1])
                fitness_score = parts[1].split(": ")[1]
                fitness_score = float(fitness_score) if fitness_score != "N/A" else None
                score = float(parts[2].split(": ")[1])
                total_profit = float(parts[3].split(": ")[1])
                total_penalty = float(parts[4].split(": ")[1])
                remaining_packages = int(parts[5].split(": ")[1])
                used_trucks = int(parts[6].split(": ")[1])
                log_data.append((iteration, fitness_score, score, total_profit, total_penalty, remaining_packages, used_trucks))
            except (IndexError, ValueError) as e:
                print(f"Fel vid bearbetning av rad: {line}, fel: {e}")

# Skapa DataFrame för analys
if not log_data:
    print("Inga data kunde läsas från loggfilen.")
else:
    df_logs = pd.DataFrame(
        log_data,
        columns=["Iteration", "Fitness Score", "Score", "Total Profit", "Total Penalty", "Remaining Packages", "Used Trucks"]
    )

    # Visualisera förbättringar
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    plt.plot(df_logs["Iteration"], df_logs["Score"], label="Score (Net Profit)")
    plt.plot(df_logs["Iteration"], df_logs["Fitness Score"], label="Fitness Score", linestyle="--")
    plt.xlabel("Iteration")
    plt.ylabel("Value")
    plt.title("Förbättringar med Optimeringsalgoritmer")
    plt.legend()
    plt.grid()
    plt.show()


Fel vid bearbetning av rad: Method: Greedy, Iteration: 0, Fitness Score: N/A, Total Profit: 14660.0, Delivered Packages: 2201, Remaining Packages: 7799, Used Trucks: 10
, fel: invalid literal for int() with base 10: 'Greedy'
Fel vid bearbetning av rad: Method: Genetic, Iteration: 1, Fitness Score: 2216394.0, Total Profit: 15027.0, Delivered Packages: 2201, Remaining Packages: 7799, Used Trucks: 10
, fel: invalid literal for int() with base 10: 'Genetic'
Fel vid bearbetning av rad: Method: Genetic, Iteration: 10, Fitness Score: 2216394.0, Total Profit: 15025.0, Delivered Packages: 2200, Remaining Packages: 7800, Used Trucks: 10
, fel: invalid literal for int() with base 10: 'Genetic'
Fel vid bearbetning av rad: Method: Genetic, Iteration: 20, Fitness Score: 2215392.0, Total Profit: 15025.0, Delivered Packages: 2200, Remaining Packages: 7800, Used Trucks: 10
, fel: invalid literal for int() with base 10: 'Genetic'
Fel vid bearbetning av rad: Method: Genetic, Iteration: 30, Fitness Score:

In [8]:
# Load the lagerstatus.csv file into a DataFrame
df_lagerstatus = pd.read_csv(lagerstatus)

# Calculate the total profit
total_profit_lagerstatus = df_lagerstatus['Vikt'].sum() / 10

print(f"Total Profit in lagerstatus.csv: {total_profit_lagerstatus}")

Total Profit in lagerstatus.csv: 6236.120000000001
